# Imports

In [7]:
import pandas as pd
from collections import Counter
import re
from torch.utils.data import Dataset
import numpy as np
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torch.utils.data import DataLoader
from transformers import AutoTokenizer
from transformers import AutoModelForPreTraining
from transformers import AutoModel 
BERTIMBAU = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')
import pyconll

Some weights of BertForPreTraining were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Preprocessing

Baseado em: https://github.com/soutsios/pos-tagger-bert/blob/master/pos_tagger_bert.ipynb

In [8]:
UD_PORTUGUESE_TRAIN = './datasets/pt_bosque-ud-train.conllu'
UD_PORTUGUESE_DEV = './datasets/pt_bosque-ud-dev.conllu'
UD_PORTUGUESE_TEST = './datasets/pt_bosque-ud-test.conllu'

In [9]:
def read_conllu(path):
    data = pyconll.load_from_file(path)
    tagged_sentences=[]
    t=0
    for sentence in data:
        tagged_sentence=[]
        for token in sentence:
            if token.upos and token.form:
                t+=1
                tagged_sentence.append((token.form.lower(), token.upos))
        tagged_sentences.append(tagged_sentence)
    return tagged_sentences

In [10]:
train_sentences = read_conllu(UD_PORTUGUESE_TRAIN)
val_sentences = read_conllu(UD_PORTUGUESE_DEV)
test_sentences = read_conllu(UD_PORTUGUESE_TEST)

In [11]:
print("Tagged sentences in train set: ", len(train_sentences))
print("Tagged words in train set:", len([item for sublist in train_sentences for item in sublist]))
print(40*'=')
print("Tagged sentences in dev set: ", len(val_sentences))
print("Tagged words in dev set:", len([item for sublist in val_sentences for item in sublist]))
print(40*'=')
print("Tagged sentences in test set: ", len(test_sentences))
print("Tagged words in test set:", len([item for sublist in test_sentences for item in sublist]))
print(40*'*')
print("Total sentences in dataset:", len(train_sentences)+len(val_sentences)+len(test_sentences))

Tagged sentences in train set:  7018
Tagged words in train set: 171776
Tagged sentences in dev set:  1172
Tagged words in dev set: 28447
Tagged sentences in test set:  1167
Tagged words in test set: 27605
****************************************
Total sentences in dataset: 9357


In [15]:
print(train_sentences[0])

[('pt', 'PROPN'), ('em', 'ADP'), ('o', 'DET'), ('governo', 'NOUN')]


In [16]:
# Some usefull functions
def tag_sequence(sentences):
    return [[t for w, t in sentence] for sentence in sentences]

def text_sequence(sentences):
    return [[w for w, t in sentence] for sentence in sentences]

# Vocab

In [28]:
class Vocab(object):
    def __init__(self, iter, max_size=None, sos_token=None, eos_token=None, unk_token=None):
        """Initialize the vocabulary.
        Args:
            iter: An iterable which produces sequences of tokens used to update
                the vocabulary.
            max_size: (Optional) Maximum number of tokens in the vocabulary.
            sos_token: (Optional) Token denoting the start of a sequence.
            eos_token: (Optional) Token denoting the end of a sequence.
            unk_token: (Optional) Token denoting an unknown element in a
                sequence.
        """
        self.max_size = max_size
        self.pad_token = '<pad>'
        self.sos_token = sos_token
        self.eos_token = eos_token
        self.unk_token = unk_token

        # Add special tokens.
        id2word = [self.pad_token]
        if sos_token is not None:
            id2word.append(self.sos_token)
        if eos_token is not None:
            id2word.append(self.eos_token)
        if unk_token is not None:
            id2word.append(self.unk_token)

        # Update counter with token counts.
        counter = Counter()
        for x in iter:
            counter.update(x)

        # Extract lookup tables.
        if max_size is not None:
            counts = counter.most_common(max_size)
        else:
            counts = counter.items()
            counts = sorted(counts, key=lambda x: x[1], reverse=True)
        words = [x[0] for x in counts]
        id2word.extend(words)
        word2id = {x: i for i, x in enumerate(id2word)}

        self._id2word = id2word
        self._word2id = word2id

    def __len__(self):
        return len(self._id2word)

    def word2id(self, word):
        """Map a word in the vocabulary to its unique integer id.
        Args:
            word: Word to lookup.
        Returns:
            id: The integer id of the word being looked up.
        """
        if word in self._word2id:
            return self._word2id[word]
        elif self.unk_token is not None:
            return self._word2id[self.unk_token]
        else:
            raise KeyError('Word "%s" not in vocabulary.' % word)

    def id2word(self, id):
        """Map an integer id to its corresponding word in the vocabulary.
        Args:
            id: Integer id of the word being looked up.
        Returns:
            word: The corresponding word.
        """
        return self._id2word[id]

# CoNLLDataset e Annotation

In [26]:
class Annotation(object):
    def __init__(self):
        """A helper object for storing annotation data."""
        self.tokens = []
        self.pos_tags = []


class CoNLLDataset(Dataset):
    def __init__(self, fname, max_exs=None):
        """Initializes the CoNLLDataset.
        Args:
            fname: The .conllu file to load data from.
        """
        self.fname = fname
        self.annotations = self.process_conll_file(fname, max_exs)
        self.token_vocab = Vocab([x.tokens for x in self.annotations],
                                 unk_token='<unk>')
        self.pos_vocab = Vocab([x.pos_tags for x in self.annotations])
        

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        annotation = self.annotations[idx]
        input = [self.token_vocab.word2id(x) for x in annotation.tokens]
        target = [self.pos_vocab.word2id(x) for x in annotation.pos_tags]
        return input, target

    def process_conll_file(self, fname, max_exs):
        # Read the entire file.
        with open(fname, 'r') as f:
            raw_text = f.read()
        # Split into chunks on blank lines.
        chunks = re.split(r'^\n', raw_text, flags=re.MULTILINE)
        #print(chunks)
        # Process each chunk into an annotation.
        annotations = []
        exs = 0
        for chunk in chunks:
            if not max_exs or exs < max_exs:
                annotation = Annotation()
                lines = chunk.split('\n')
                # Iterate over all lines in the chunk.
                for line in lines:
                    # If line is empty ignore it.
                    if len(line)==0:
                        continue
                    # If line is a commend ignore it.
                    if line[0] == '#':
                        continue
                    # Otherwise split on tabs and retrieve the token and the
                    # POS tag fields.
                    fields = line.split('\t')
                    annotation.tokens.append(fields[1])
                    annotation.pos_tags.append(fields[3])
                if (len(annotation.tokens) > 0) and (len(annotation.pos_tags) > 0):
                    annotations.append(annotation)
            exs += 1
        return annotations

In [29]:
# Load datasets.
train_dataset = CoNLLDataset('./datasets/pt_bosque-ud-train.conllu', 4096)
dev_dataset = CoNLLDataset('./datasets/pt_bosque-ud-dev.conllu', 1024)

In [30]:
train_dataset[0]

([235, 19, 7, 5, 75], [6, 7, 3, 2, 1])

# Funções: pad() e collate_annotations()

In [25]:
def pad(sequences, max_length, pad_value=0):
    """Pads a list of sequences.
    Args:
        sequences: A list of sequences to be padded.
        max_length: The length to pad to.
        pad_value: The value used for padding.
    Returns:
        A list of padded sequences.
    """
    out = []
    for sequence in sequences:
        padded = sequence + [0]*(max_length - len(sequence))
        out.append(padded)
    return out


def collate_annotations(batch):
    """Function used to collate data returned by CoNLLDataset."""
    # Get inputs, targets, and lengths.
    inputs, targets = zip(*batch)
    lengths = [len(x) for x in inputs]
    # Sort by length.
    sort = sorted(zip(inputs, targets, lengths),
                  key=lambda x: x[2],
                  reverse=True)
    inputs, targets, lengths = zip(*sort)
    # Pad.
    max_length = max(lengths)
    inputs = pad(inputs, max_length)
    targets = pad(targets, max_length)
    # Transpose.
    inputs = list(map(list, zip(*inputs)))
    targets = list(map(list, zip(*targets)))
    # Convert to PyTorch variables.
    inputs = Variable(torch.LongTensor(inputs))
    targets = Variable(torch.LongTensor(targets))
    lengths = Variable(torch.LongTensor(lengths))
    if torch.cuda.is_available():
        inputs = inputs.cuda()
        targets = targets.cuda()
        lengths = lengths.cuda()
    return inputs, targets, lengths

In [ ]:
collate_annotations

# Baseado em: https://pytorch.org/hub/huggingface_pytorch-transformers/

In [39]:
import torch
# tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased', do_lower_case=False)

text_1 = "Who was Jim Henson ?"
text_2 = "Jim Henson was a puppeteer"

# Tokenized input with special tokens around it (for BERT: [CLS] at the beginning and [SEP] at the end)
indexed_tokens = tokenizer.encode(text_1, text_2, add_special_tokens=True)

Downloading: 100%|████████| 43.0/43.0 [00:00<00:00, 9.49kB/s]
Downloading: 100%|█████████| 205k/205k [00:00<00:00, 577kB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 762B/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 33.2kB/s]


In [43]:
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
segments_tensors = torch.tensor([segments_ids])
tokens_tensor = torch.tensor([indexed_tokens])

# model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')
model = AutoModelForPreTraining.from_pretrained('neuralmind/bert-base-portuguese-cased')

with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, token_type_ids=segments_tensors)

Some weights of BertForPreTraining were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Token classification (HuggingFace)
https://huggingface.co/docs/transformers/tasks/token_classification
https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification.ipynb#scrollTo=MOsHUjgdIrIW

In [2]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

## Loading the dataset

In [3]:
from datasets import load_dataset, load_metric

In [4]:
datasets = load_dataset("conll2003")

Dataset conll2003 downloaded and prepared to /home/danieldasilvacosta/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


100%|█████████████████████████| 3/3 [00:00<00:00, 274.26it/s]


In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

In [6]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [8]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [9]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [31]:
show_random_elements(datasets["train"])

,id,tokens,pos_tags,chunk_tags,ner_tags
0,8565,"[FLORIDA, AT, ST, LOUIS]","[NNP, NNP, NNP, NNP]","[B-NP, I-NP, I-NP, I-NP]","[B-ORG, O, B-LOC, I-LOC]"
1,546,"[Attendance, :, 3,000]","[NN, :, CD]","[B-NP, O, B-NP]","[O, O, O]"
2,7043,"[Alex, O'Brien, (, U.S., ), beat, Nicolas, Lapentti, (, Ecuador, ), 6-4, 1-6, 6-4, 6-3]","[NNP, NNP, (, NNP, ), VB, NNP, NNP, (, NNP, ), CD, CD, CD, CD]","[B-NP, I-NP, O, B-NP, O, B-VP, B-NP, I-NP, O, B-NP, O, B-NP, I-NP, I-NP, I-NP]","[B-PER, I-PER, O, B-LOC, O, O, B-PER, I-PER, O, B-LOC, O, O, O, O, O]"
3,10614,"[SOCCER, -, GERMAN, FIRST, DIVISION, SUMMARIES, .]","[NN, :, NNP, NNP, NNP, NNP, .]","[B-NP, O, B-NP, I-NP, I-NP, I-NP, O]","[O, O, B-MISC, O, O, O, O]"
4,1821,"[Kiev, 's, Foreign, Minister, Hennady, Udovenko, said, Beijing, was, overreacting, .]","[NNP, POS, NNP, NNP, NNP, NNP, VBD, NNP, VBD, VBG, .]","[B-NP, B-NP, I-NP, I-NP, I-NP, I-NP, B-VP, B-NP, B-VP, I-VP, O]","[B-LOC, O, O, O, B-PER, I-PER, O, B-LOC, O, O, O]"
5,11663,"[The, unexpectedly, weak, figures, convinced, markets, that, Japanese, interest, rates, would, stay, at, rock-bottom, levels, --, weakening, the, yen, .]","[DT, RB, JJ, NNS, VBN, NNS, IN, JJ, NN, NNS, MD, VB, IN, JJ, NNS, :, VBG, DT, NN, .]","[B-NP, I-NP, I-NP, I-NP, B-VP, B-NP, B-SBAR, B-NP, I-NP, I-NP, B-VP, I-VP, B-PP, B-NP, I-NP, O, B-VP, B-NP, I-NP, O]","[O, O, O, O, O, O, O, B-MISC, O, O, O, O, O, O, O, O, O, O, O, O]"
6,13920,"[Rubin, was, widely, hailed, as, the, architect, of, the, dollar, 's, comeback, ,, using, skills, and, expertise, gained, in, 26, years, on, Wall, Street, ,, part, of, which, were, spent, as, co-chairman, of, Goldman, ,, Sachs, and, Co, .]","[NNP, VBD, RB, VBN, IN, DT, NN, IN, DT, NN, POS, NN, ,, VBG, NNS, CC, NN, VBD, IN, CD, NNS, IN, NNP, NNP, ,, NN, IN, WDT, VBD, VBN, IN, NN, IN, NNP, ,, NNP, CC, NNP, .]","[B-NP, B-VP, I-VP, I-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, B-NP, I-NP, O, B-VP, B-NP, I-NP, I-NP, B-VP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, O, B-NP, B-PP, B-NP, O, B-VP, B-PP, B-NP, B-PP, B-NP, I-NP, I-NP, I-NP, I-NP, O]","[B-PER, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-LOC, I-LOC, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG]"
7,12918,"[Congress, is, constitutionally, obliged, to, approve, the, budget, by, the, end, of, year, but, regularly, fails, to, meet, that, requirement, .]","[NNP, VBZ, RB, JJ, TO, VB, DT, NN, IN, DT, NN, IN, NN, CC, RB, VBZ, TO, VB, DT, NN, .]","[B-NP, B-VP, B-NP, B-ADJP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, O, B-VP, I-VP, I-VP, I-VP, B-NP, I-NP, O]","[B-ORG, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
8,11195,"[Bulgaria, has, restored, ownership, rights, to, pre-communist, private, owners, of, 75, percent, of, the, arable, land, or, around, four, million, hectares, ,, an, Agriculture, Ministry, official, said, on, Wednesday, .]","[NNP, VBZ, VBN, NN, NNS, TO, VB, JJ, NNS, IN, CD, NN, IN, DT, JJ, NN, CC, IN, CD, CD, NNS, ,, DT, NNP, NNP, NN, VBD, IN, NNP, .]","[B-NP, B-VP, I-VP, B-NP, I-NP, B-VP, I-VP, B-NP, I-NP, B-PP, B-NP, I-NP, B-PP, B-NP, I-NP, I-NP, I-NP, B-PP, B-NP, I-NP, I-NP, O, B-NP, I-NP, I-NP, I-NP, B-VP, B-PP, B-NP, O]","[B-LOC, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-ORG, I-ORG, O, O, O, O, O]"
9,8971,"[During, his, visit, to, Slovenia, ,, Kwasniewski, is, also, scheduled, to, meet, Prime, Minister, Janez, Drnovsek, ,, representatives, of, Slovenian, political, parties, and, representatives, of, the, Chamber, of, Economy, .]","[IN, PRP$, NN, TO, NNP, ,, NNP, VBZ, RB, VBN, TO, VB, NNP, NNP, NNP, NNP, ,, NNS, IN, JJ, JJ, NNS, CC, NNS, IN, DT, NNP, IN, NNP, .]","[B-PP, B-NP, I-NP, B-PP, B-NP, O, B-NP, B-VP, I-VP, I-VP, I-VP, I-VP, B-NP, I-NP, I-NP, I-NP, O, B-NP, B-PP, B-NP, I-NP, I-NP, O, B-NP, B-PP, B-NP, I-NP, B-PP, B-NP, O]","[O, O, O, O, B-LOC, O, B-PER, O, O, O, O, O, O, O, B-PER, I-PER, O, O, O, B-MISC, O, O, O, O, O, O, B-ORG, I-ORG, I-ORG, O]"


## Preprocessing the data

In [32]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading: 100%|████████| 28.0/28.0 [00:00<00:00, 17.9kB/s]
Downloading: 100%|███████████| 483/483 [00:00<00:00, 180kB/s]
Downloading: 100%|█████████| 226k/226k [00:00<00:00, 484kB/s]
Downloading: 100%|█████████| 455k/455k [00:00<00:00, 640kB/s]


In [33]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [34]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [35]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 2023, 2003, 2028, 6251, 3975, 2046, 2616, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [36]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [37]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


In [38]:
len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])

(31, 39)

In [39]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 10, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 18, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, None]


In [40]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


In [42]:
label_all_tokens = True

In [43]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]): # class (pos_tags ou chunk_tags ou ner_tags)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [44]:
tokenize_and_align_labels(datasets['train'][:5])

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102], [101, 1996, 2647, 3222, 2056, 2006, 9432, 2009, 18335, 2007, 2446, 6040, 2000, 10390, 2000, 18454, 2078, 2329, 12559, 2127, 6529, 5646, 3251, 5506, 11190, 4295, 2064, 2022, 11860, 2000, 8351, 1012, 102], [101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100], [-100, 5, 0, 

In [46]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /home/danieldasilvacosta/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-bd9cc90169c59e2a.arrow
Loading cached processed dataset at /home/danieldasilvacosta/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-23e607d2d5ae4965.arrow
Loading cached processed dataset at /home/danieldasilvacosta/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-115bf85215420fa9.arrow


In [49]:
datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [59]:
tokenized_datasets["train"][0]

{'id': '0',
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

## Fine-tuning the model

In [51]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Downloading: 100%|████████| 256M/256M [00:12<00:00, 21.2MB/s]
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['clas

In [53]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01
)

In [54]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [55]:
metric = load_metric("seqeval")

In [56]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'PER': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [57]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [60]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [61]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, id, chunk_tags, ner_tags, pos_tags. If tokens, id, chunk_tags, ner_tags, pos_tags are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/home/danieldasilvacosta/dev/doutorado/uff--trabalho-2--2022/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14042
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results